# Polarizability tensor calculation using BigDFT

The polarizability tensor represents the response of the charges of a system (its dipole) to the application of an external electric field. 

To compute this polarizability tensor, 6 BigDFT calculations where the systems is subject to an external electric field are performed: 2 calculations (one with a positive electric field amplitude, the other with a negative amplitude) along each space coordinate ($x$, $y$ and $z$). The elements of the polarizability tensor are then defined by the ratio of delta of the dipole in one direction and the delta of the electric field amplitudes.

**Import the relevant classes**

In [1]:
from __future__ import print_function
from inputfiles import Posinp, Input
from poltensor import PolTensorCalc

## The Posinp class

A class Posinp to represent the input geometry files of BigDFT was implemented. A posinp instance is read from a file, given its name:

In [2]:
pos_ref = Posinp("posinp.xyz")
# When you print the Posinp instance, it returns the content of the posinp
print(pos_ref)

2   angstroem
free
N    2.976307744763e-23    6.872205902435e-23    0.0107162001879
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769



Note that the posinp may also be initialized from a string by using the class method `from_string`:

In [3]:
# Turn the reference posinp into a string
str_pos_ref = str(pos_ref)
# Initialize another posinp from the string
pos_from_str = Posinp.from_string(str_pos_ref)
# Check that it corresponds to the same posinp
print(pos_from_str)

2   angstroem
free
N    2.976307744763e-23    6.872205902435e-23    0.0107162001879
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769



This posinp can also be written in a file, here `ref.xyz`:

In [4]:
pos_ref.write("ref.xyz")

We can also check that this posinp is the same as the initial one (the output must be `True`):

In [5]:
pos = Posinp("ref.xyz")
print(pos == pos_ref)

True


The attributes of a Posinp instance are:

* `n_at`: gives the number of atoms of the system as an integer

* `BC`: gives the boundary condition as a string

* `units`: the units of the atom coordinates as a string

* `atoms`: the atoms of the system as a list of dictionaries. Each dictionary gives the position of the atom and its element (`'Type'`).

In [6]:
print("Number of atoms:", pos_ref.n_at)
print("Boundary conditions:", pos_ref.BC)
print("Units:", pos_ref.units)
print("Atoms:", pos_ref.atoms)

Number of atoms: 2
Boundary conditions: free
Units: angstroem
Atoms: [{'Position': [2.976307744763e-23, 6.872205902435e-23, 0.0107162001879], 'Type': 'N'}, {'Position': [-1.104344885754e-23, -4.873421785298e-23, 1.104273795769], 'Type': 'N'}]


## The Input class

Another class Input is used to represent the BigDFT input files. It inherits from the `dict` class. An input file is therefore initialized from a dictionary, but class methods allow to initialize it from a string (using the `Input.from_string` classmethod, in a similar manner as the `Posinp.from_string` presented above) and also from a file, through the class method `Input.from_file`:

In [7]:
input_ref = Input.from_file("input.yaml")
print(input_ref)

{'dft': {'rmult': [5.0, 7.0], 'elecfield': [0.0001, 0.0, 0.0], 'hgrids': [0.45, 0.45, 0.45], 'gnrm_cv': 0.0001}}


As mentionned, an input file can be initialized from a string:

In [8]:
string = """\
 dft:
   rmult: [9.0, 9.0]
   hgrids: [0.35, 0.35, 0.35]"""

input_from_string = Input.from_string(string)
print(input_from_string)

{'dft': {'rmult': [9.0, 9.0], 'hgrids': [0.35, 0.35, 0.35]}}


You can also easily write an input file:

In [9]:
input_ref.write("ref.yaml")

# Check that the file written is the same as the previous one
yaml_file = Input.from_file("ref.yaml")
print(input_ref == yaml_file)

True


## Find the polarizability tensor of the given geometry of the system: the PolTensorCalc class

The `PolTensorCalc` class was implemented to simplify the polarizability tensor calculation, as will be shown in the rest of this notebook.

### Prepare a calculation with small electric field values

The PolTensorCalc class requires an input file and a posinp to be initialized. If no electric field amplitudes are passed, then a default value is given. You can also define `folder` to specify the folder where you want the calculation to be run (here, in a folder named `pol_tensor`, that will be created when the calculation is actually run).

In [10]:
# Read the input file and the initial geometry and print them
input_yaml = Input.from_file("input.yaml")
posinp = Posinp("posinp.xyz")
print(input_yaml)
print(posinp)

# Define the amplitude of the electric field along each of the three space coordinates
ef_a = [0.0001]*3

# Initialize the calculation
ptc = PolTensorCalc(input_yaml, posinp, ef_amplitudes=ef_a, folder="pol_tensor")

{'dft': {'rmult': [5.0, 7.0], 'elecfield': [0.0001, 0.0, 0.0], 'hgrids': [0.45, 0.45, 0.45], 'gnrm_cv': 0.0001}}
2   angstroem
free
N    2.976307744763e-23    6.872205902435e-23    0.0107162001879
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769



### Run the calculation

You can finally run the calculation using the `run` method. It has three optional arguments:

* `nmpi`: number of MPI tasks as an init

* `nomp`: number of OpenMP tasks as an int

* `force_run`: boolean setting is the calculations must be run even though there already exists a logfile corresponding to the same calculation.

In [11]:
# We use 4 OpenMP tasks here
ptc.run(nomp=4)

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_x+_0.0001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_x-_0.0001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_y+_0.0001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_y-_0.0001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_z+_0.0001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_z-_0.0001
Logfile log.yaml already exists!



### Get the polarizability tensor

After the calculation is run, there is a post-processing step initializing the attribute `pol_tensor`. The value of this attribute is the polarizability tensor obtained from the 6 calculations. The polarizability tensor is in atomic units ($e^2 {a_o}^2 / E_h$). You must multiply its elements by `1.6487772E^-41` to convert them into SI units (C.m$^2$.V$^{-1}$) or by 0.1481847 to get them in $\unicode[serif]{xC5}^3$.

In [12]:
ptc.pol_tensor

array([[  1.05465000e+01,  -3.00000000e-04,  -3.50000000e-04],
       [ -3.00000000e-04,   1.05465000e+01,  -3.50000000e-04],
       [  0.00000000e+00,   0.00000000e+00,   1.50236000e+01]])

As expected, the polarizability tensor is diagonal and $\alpha_{xx} = \alpha_{yy} < \alpha_{zz}$. The values are consistent with what is reported in the litterature and the experiment (see Table II of [George Maroulis and Ajit J. Thakkar, *J. Chem. Phys.* **88**, 7623 (1988)](http://dx.doi.org/10.1063/1.454327)).

With a larger electric field, the results should be more precise (the polarizability tensor elements only being computed as the slope between the dipole and the electric field by using two points). 

**Be carefull**: if the electric fields applied are high enough for some electrons to escape the system, then you cannot trust the results anymore. In such a case, the system you are trying to simulate is not the one you expected, because an occupied orbital has a positive energy under the perturbation of the elctric field. There is actually a rule of thumb to know if the electric field applied along a direction is not too high: the difference of potential along the electric field (*i.e.* the amplitude of the electric field times the length of the BigDFT grid along the direction of the electric field) has to be roughly two times smaller than the absolute value of the HOMO when there is no perturbation.

**Notes: **

* it might be nice to implement that check: only needs to check that all occupied orbitals are negative. A warning may be printed or an error raised if one occupied orbital is above some threshold (say, -0.1 eV for the warning, rather than 0.0 eV for the error).

* Another possibility would be to add an optional argument `ref` to the `__init__` method of PolarizabilityTensorCalculation. This way, the check occurs before running any electric field calculation. The energy of the HOMO without perturbation can be found thanks to the reference logfile. The length of the grid is also known (it must be the same as the one used for the electric field calculation, though; maybe use bigdft-tool to make sure that we have the lengths of the simulation grid of the electric field calculations).

## Same calculation, with a larger electric field amplitude

Let us keep the same input files but use larger electric field amplitudes to run another polarizability tensor calculation in another folder. This is done in a very few lines of code:

In [13]:
ef_a_2 = [0.001]*3
ptc2 = PolTensorCalc(input_yaml, posinp, ef_amplitudes=ef_a_2, folder="pol_tensor_large_EF")
ptc2.run(nomp=4)
ptc2.pol_tensor

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_x+_0.001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_x-_0.001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_y+_0.001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_y-_0.001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_z+_0.001
Logfile log.yaml already exists!

/Users/maximemoriniere/post-doc/bigdft/bigdft/build/calculs/Raman/along_z-_0.001
Logfile log.yaml already exists!



array([[  1.05455000e+01,  -3.30000000e-04,  -3.35000000e-04],
       [ -3.30000000e-04,   1.05455000e+01,  -3.35000000e-04],
       [ -3.50000000e-05,  -3.50000000e-05,   1.50230000e+01]])

As you can see, this does not really affect the results here. This is due to the fact that the smaller electric field amplitudes were already high enough to get a good "signal" (remember that we evaluate the delta of the dipoles, and this delta must be high enough to be captured). This is also due to the fact that the underlying wavelet grid was dense and large enough and that the convergence parameter `gnrm_cv` was low enough for the "noise" level to bas as low as possible. That is why you must ensure that the results are converged with respect to these parameters. 